In [2]:
import numpy as np
import h5py



In [7]:
path_hydro= "/net/hypernova/data2/FLAMINGO/L1000N1800/HYDRO_FIDUCIAL/SOAP-HBT/halo_properties_0077.hdf5"

with h5py.File(path_hydro, "r") as handle:
    # TotalMass = handle["ExclusiveSphere/100kpc/TotalMass"][...]
    # StellarMass = handle["ExclusiveSphere/100kpc/StellarMass"][:] ## NOT USED IN SOWMYA's CODE
    # COMstellarvelocity = handle["ExclusiveSphere/100kpc/StellarCentreOfMassVelocity"][:] ## NOT USED IN SOWMYA's CODE
    # COMvelocity = handle["ExclusiveSphere/100kpc/CentreOfMassVelocity"][:]
    # Trackid = handle["InputHalos/HBTplus/TrackId"][:]
    # HOSTFOFID = handle["InputHalos/HBTplus/HostFOFId"][:]
    # HaloCatalogueIndex = handle["InputHalos/HaloCatalogueIndex"][:]
    # HOSTHALOINDEX = handle["SOAP/HostHaloIndex"][:]
    # FOFMass = handle["InputHalos/FOF/Masses"][:]
    # NoofBoundParticles = handle["InputHalos/NumberOfBoundParticles"][:]
    # NoofDMParticles = handle["ExclusiveSphere/100kpc/NumberOfDarkMatterParticles"][:]
    COM = handle["ExclusiveSphere/100kpc/CentreOfMass"][...]
    IsCentral = handle["InputHalos/IsCentral"][:]


In [11]:
str(14.5).replace('.','_')

'14_5'

In [ ]:

with h5py.File('/disks/cosmodm/vdvuurst/data/velocity_data_M14_14_5_trial_largersecondary_newquantity.h5','r') as f:
    data = f['masses'][:]



AttributeError: 'tuple' object has no attribute 'shape'

In [15]:
data.shape[0] - 767610320

47055